<a href="https://colab.research.google.com/github/akshatdalton/simple-text-search/blob/main/Jina_Simple_Neural_Search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install -U jina

In [ ]:
import numpy as np
from jina import DocumentArray, Executor, requests


class CharEmbed(Executor):  # a simple character embedding with mean-pooling
    offset = 32  # letter `a`
    dim = 127 - offset + 1  # last pos reserved for `UNK`
    char_embd = np.eye(dim) * 1  # one-hot embedding for all chars

    @requests
    def foo(self, docs: DocumentArray, **kwargs):
        for d in docs:
            r_emb = [ord(c) - self.offset if self.offset <= ord(c) <= 127 else (self.dim - 1) for c in d.text]
            d.embedding = self.char_embd[r_emb, :].mean(axis=0)  # average pooling

In [ ]:
from jina import DocumentArray, Executor, requests


class Indexer(Executor):
    _docs = DocumentArray()  # for storing all documents in memory

    @requests(on='/index')
    def foo(self, docs: DocumentArray, **kwargs):
        self._docs.extend(docs)  # extend stored `docs`

    @requests(on='/search')
    def bar(self, docs: DocumentArray, **kwargs):
        docs.match(self._docs, metric='euclidean', limit=20)

In [ ]:
from jina import Flow

f = (Flow(port_expose=12345, protocol='http', cors=True)
        .add(uses=CharEmbed, replicas=2)
        .add(uses=Indexer))  # build a Flow, with 2 shard CharEmbed, tho unnecessary

In [ ]:
from jina import Document

data = ["Hello World", "Hello Universe", "World Hello!", "Universe Hello!"]

from jina.types.request import Response


def print_matches(resp: Response):  # the callback function invoked when task is done
    for idx, d in enumerate(resp.docs[0].matches[:3]):  # print top-3 matches
        print(f'[{idx}]{d.scores["euclidean"].value:2f}: "{d.text}"')

with f:
    f.post('/index', (Document(text=t.strip()) for t in data if t.strip()))  # index all lines of _this_ file
    # f.block()  # block for listening request
    f.post('/search', Document(text=input("Enter your query\n")), on_done=print_matches)

⠸ 2/3 waiting gateway to be ready...        gateway@954[W]:CORS is enabled. This service is now accessible from any website!
           Flow@61[I]:🎉 Flow is ready to use!
	🔗 Protocol: 		HTTP
	🏠 Local access:	0.0.0.0:12345
	🔒 Private network:	172.28.0.2:12345
	🌐 Public address:	35.245.158.192:12345
	💬 Swagger UI:		http://localhost:12345/docs
	📚 Redoc:		http://localhost:12345/redoc


Enter your query
Universe
[0]0.232829: "Hello Universe"
[1]0.241810: "Universe Hello!"
[2]0.482327: "World Hello!"
